In [1]:
# libary imports
import geopandas as gpd
import pandas as pd



c:\Users\b8008458\Anaconda3\envs\environment\lib\site-packages\pyproj\__init__.py:91: UserWarning: Valid PROJ data directory not found. Either set the path using the environmental variable PROJ_LIB or with `pyproj.datadir.set_data_dir`.
  warnings.warn(str(err))
